<a href="https://colab.research.google.com/github/zsowen/Project_3/blob/main/Jake/analyst_ratings_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Sentiment Analysis w Big Data

In [2]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 5.1 MB/s 


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [4]:
# load data
from google.colab import files
files.upload()

Saving analyst_ratings_processed.csv to analyst_ratings_processed.csv


In [26]:
#
df_analyst_ratings_processed = pd.read_csv('analyst_ratings_processed.csv')
df_analyst_ratings_processed

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A
...,...,...,...,...
1400464,1413844.0,Top Narrow Based Indexes For August 29,2011-08-29 10:41:00-04:00,ZX
1400465,1413845.0,Recap: Wednesday's Top Percentage Gainers and ...,2011-06-22 16:44:00-04:00,ZX
1400466,1413846.0,UPDATE: Oppenheimer Color on China Zenix Auto ...,2011-06-21 08:26:00-04:00,ZX
1400467,1413847.0,Oppenheimer Initiates China Zenix At Outperfor...,2011-06-21 05:59:00-04:00,ZX


In [30]:
df_analyst_ratings_processed.drop(['date', 'Unnamed: 0'], axis=1)

,title,stock
0,Stocks That Hit 52-Week Highs On Friday,A
1,Stocks That Hit 52-Week Highs On Wednesday,A
2,71 Biggest Movers From Friday,A
3,46 Stocks Moving In Friday's Mid-Day Session,A
4,B of A Securities Maintains Neutral on Agilent...,A
...,...,...
1400464,Top Narrow Based Indexes For August 29,ZX
1400465,Recap: Wednesday's Top Percentage Gainers and ...,ZX
1400466,UPDATE: Oppenheimer Color on China Zenix Auto ...,ZX
1400467,Oppenheimer Initiates China Zenix At Outperfor...,ZX


In [31]:
df_analyst_ratings_processed.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,,A
2,2.0,71 Biggest Movers From Friday,,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,,A


In [32]:
analyst_ratings_processed= df_analyst_ratings_processed.groupby(df_analyst_ratings_processed['stock'])['title'].apply(lambda x: ','.join(x)).reset_index()
# test_df= Prices_with_changes.groupby(Prices_with_changes['Symbol'])['title'].apply(lambda x: ','.join(x)).agg({
#     'Opening_avg': 'mean',
#     'Closing_avg': 'mean'
# }).reset_index()

analyst_ratings_processed

,stock,title
0,A,"Stocks That Hit 52-Week Highs On Friday,Stocks..."
1,AAMC,Altisource Asset Mgmt Q1 EPS $2.350 Up From $0...
2,AAME,Atlantic American Q1 EPS $(0.400) Down From $0...
3,AAN,Shares of retail & apparel companies are tradi...
4,AAOI,"96 Biggest Movers From Friday,70 Stocks Moving..."
...,...,...
6187,ZTR,"Stocks That Hit 52-Week Lows On Thursday,Stock..."
6188,ZTS,Shares of several healthcare companies are tra...
6189,ZU,"CBP Taps Nine Firms For E-Commerce Data Pilot,..."
6190,ZUMZ,40 Stocks Moving in Friday's Pre-Market Sessio...


In [33]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

big_ol_text_blob = analyst_ratings_processed['title']

big_ol_text_blob = big_ol_text_blob.tolist()
big_ol_text_blob = big_ol_text_blob
#big_ol_text_blob = ''.join(big_ol_text_blob)

#big_ol_text_blob = [big_ol_text_blob]


In [ ]:
big_ol_text_blob

In [35]:
# #Clean the data

clean_headlines = []

for i in range(0, len(big_ol_text_blob)):
  clean_headlines.append(re.sub("b[(')]+", '', big_ol_text_blob[i] ))
  clean_headlines[i] = re.sub('b[(")]+', '', clean_headlines[i] )
  clean_headlines[i] = re.sub("\'", '', clean_headlines[i] )
  clean_headlines[i] = re.sub("‚Ç¨", '', clean_headlines[i] )


In [36]:
# clean_headlines

In [37]:
#Add the clean headlines to the data set
analyst_ratings_processed['title'] = clean_headlines

In [38]:
analyst_ratings_processed.head()

,stock,title
0,A,"Stocks That Hit 52-Week Highs On Friday,Stocks..."
1,AAMC,Altisource Asset Mgmt Q1 EPS $2.350 Up From $0...
2,AAME,Atlantic American Q1 EPS $(0.400) Down From $0...
3,AAN,Shares of retail & apparel companies are tradi...
4,AAOI,"96 Biggest Movers From Friday,70 Stocks Moving..."


In [39]:
# Create a function to get the subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

In [40]:
# Create a function to get the polarity
def getPolarity(text):
  return  TextBlob(text).sentiment.polarity

In [41]:
# Create two new columns 'Subjectivity' & 'Polarity'
analyst_ratings_processed['Subjectivity'] =analyst_ratings_processed['title'].apply(getSubjectivity)
analyst_ratings_processed['Polarity'] =analyst_ratings_processed['title'].apply(getPolarity)

In [42]:
analyst_ratings_processed.loc[analyst_ratings_processed.Polarity >= 0, 'Polarity'] = 1
analyst_ratings_processed.loc[analyst_ratings_processed.Polarity < 0, 'Polarity'] = 0
analyst_ratings_processed

,stock,title,Subjectivity,Polarity
0,A,"Stocks That Hit 52-Week Highs On Friday,Stocks...",0.274279,1.0
1,AAMC,Altisource Asset Mgmt Q1 EPS $2.350 Up From $0...,0.199134,0.0
2,AAME,Atlantic American Q1 EPS $(0.400) Down From $0...,0.151984,1.0
3,AAN,Shares of retail & apparel companies are tradi...,0.424380,1.0
4,AAOI,"96 Biggest Movers From Friday,70 Stocks Moving...",0.388640,1.0
...,...,...,...,...
6187,ZTR,"Stocks That Hit 52-Week Lows On Thursday,Stock...",0.632082,0.0
6188,ZTS,Shares of several healthcare companies are tra...,0.358096,1.0
6189,ZU,"CBP Taps Nine Firms For E-Commerce Data Pilot,...",0.392805,1.0
6190,ZUMZ,40 Stocks Moving in Fridays Pre-Market Session...,0.399220,1.0


In [43]:
analyst_ratings_processed.loc[analyst_ratings_processed.Polarity == 0]

,stock,title,Subjectivity,Polarity
1,AAMC,Altisource Asset Mgmt Q1 EPS $2.350 Up From $0...,0.199134,0.0
14,ABAC,"56 Biggest Movers From Yesterday,41 Stocks Mov...",0.238110,0.0
20,ABCD,Cambium Learning Group Q3 EPS $0.12 Down From ...,0.303809,0.0
52,ACMP,Williams Partners and Access Midstream Partner...,0.245833,0.0
57,ACPW,"Active Power Shares Resume, Now Up Nearly 40%,...",0.521545,0.0
...,...,...,...,...
6163,ZAZA,ZaZa Energy Reports Extinguishment of $50.1M D...,0.335446,0.0
6164,ZBB,ZBB Energy Signs Power Purchase Agreement With...,0.347503,0.0
6175,ZIV,"ICYMI: The Inverse VIX ETN Bloodbath,3 Other I...",0.228662,0.0
6182,ZOES,Zoes Kitchen Shares Indicated Up 1% Premarket ...,0.357134,0.0


In [44]:
analyst_ratings_processed= analyst_ratings_processed.round(4)


In [45]:
# check_df= merged_df.drop(['Symbol'],1)
# check_df= check_df.drop(['title'],1)
# check_df

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')        #add string in later
cv= CountVectorizer(stop_words='english', ngram_range= (1,1), tokenizer= token.tokenize)
text_counts= cv.fit_transform(analyst_ratings_processed['title'])

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_counts, analyst_ratings_processed.Polarity, test_size=0.2, random_state = 69)


In [48]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)


In [49]:
predictions = model.predict(x_test)
predictions

array([1., 1., 1., ..., 1., 1., 0.])

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.19      0.20      0.20       143
         1.0       0.89      0.89      0.89      1096

    accuracy                           0.81      1239
   macro avg       0.54      0.54      0.54      1239
weighted avg       0.81      0.81      0.81      1239



In [53]:
# Saving relevant df as csv 
analyst_ratings_processed.to_csv(r'C:\Users\burns\OneDrive\Desktop\Code_Class\Homework\Project_3\Jake\Sentiment_analysis_6000stock.csv', encoding='utf-8', index=False)